In [8]:
"""
fetch_fundamentals.py

Fetches fundamental indicators for a list of tickers using Financial Modeling Prep API
and stores the results in PostgreSQL.
"""

import os
import requests
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
DATABASE_URL = os.getenv("DATABASE_URL")
FMP_API_KEY = os.getenv("FMP_API_KEY")

# Setup DB connection
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine)
session = Session()

def fetch_fundamentals(ticker):
    url = f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit=100&apikey={FMP_API_KEY}"
    response = requests.get(url)
    data = response.json()
    df = pd.DataFrame(data)

    if df.empty:
        print(f"No fundamentals for {ticker}")
        return None

    df["ticker"] = ticker
    df["fetched_at"] = datetime.now()
    return df

# Tickers to fetch
tickers = ["NVDA"]

all_data = []
for ticker in tickers:
    df = fetch_fundamentals(ticker)
    if df is not None:
        all_data.append(df)

if all_data:
    final_df = pd.concat(all_data)
    final_df.to_sql("fundamental_data", engine, if_exists="replace", index=False)
    print("✅ Fundamental data saved to 'fundamental_data' table in PostgreSQL.")
else:
    print("⚠️ No fundamental data retrieved.")


✅ Fundamental data saved to 'fundamental_data' table in PostgreSQL.
